## Modelado de datos para Funciones de Machine Learning

### Importando librerías 

In [8]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq


### Modelo de recomendación Item_Item

Un modelo de recomendación `item-item` basado en la similitud del coseno es un tipo de sistema de recomendación que sugiere elementos o ítems similares a aquellos que un usuario ha interactuado o mostrado interés previamente. Este enfoque se basa en la idea de que si a un usuario le gustó un cierto ítem, es probable que también le gusten ítems similares.
El perfil de un ítem generalmente se representa como un vector que contiene información relevante sobre ese ítem. Por ejemplo, en un sistema de recomendación de películas, el perfil de una película podría incluir información sobre el género, el director, el elenco de actores, las etiquetas asociadas, etc.

#### Se inicia con la creación de dummies , para luego crear la matriz necesaria para el  analisis de similitud de coseno

In [10]:
modelo_item= pd.read_parquet("data/modelo_item.parquet")
modelo_item.head(3)

,id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie


Se crea un dataframe que contenga en sus columnas los generos que puedan combinarse con los nombres de los juegos en cada una de las filas 

In [11]:
#Creacíon de dummies
modelo_item= pd.get_dummies(modelo_item, columns=["genres"], prefix="")

modelo_item= modelo_item.groupby(["id","app_name"]).sum().reset_index()

modelo_item.head(4)

,id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calcular la similitud del coseno

Se calcula la similitud del coseno entre todos los pares posibles de ítems en función de sus perfiles. La similitud del coseno mide el ángulo entre dos vectores en un espacio multidimensional y varía de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más parecidos se considerarán.

In [12]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])

In [13]:
similitudes.shape

(28850, 28850)

In [14]:
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [40]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcular la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [41]:
recomendacion_juego(308164)

18000        Santa Rockstar
17998                  BIOS
17997                  BIOS
17996    Santa Rockstar OST
17995    Santa Rockstar OST
Name: app_name, dtype: object

Modelo para render

In [66]:
modelo_item

,id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28846,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28847,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28848,2028103,Assassin’s Creed® III Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego = modelo_item[modelo_item['id'] == id]
    # devolver error en caso de vacio
    if juego.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    indices_similares = sorted(enumerate(cosine_similarity(modelo_item.iloc[[juego]], modelo_item).flatten()), key=lambda x: x[1], reverse=True)[1:6]
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_similares]['app_name']
    #Obtengo un diccionario para la salida en la api
    #juegos_recomendados=juegos_recomendados.to_dict()
    return {
        "Juegos_recomendados" :juegos_recomendados}

In [68]:
recomendacion_juego(308164)

ValueError: invalid literal for int() with base 10: '"Lethargic Sentience" Wand'

Se divide a la mitad el dataframe a fin de lograr que se pueda montar el modelo en render 

In [61]:
#Cuento filas
cant_filas= len(modelo_item)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
modelo_render= modelo_item.iloc[:mitad_filas]

In [62]:
modelo_render.shape

(2885, 24)

Almaceno el dataframe acotado 

In [63]:
modelo_render.to_parquet("data/modelo_render.parquet")

Se prueba la funcion nuevamente

In [64]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego = modelo_render[modelo_render['id'] == id]
    # devolver error en caso de vacio
    if juego.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    

    
    # Calcular la matriz de similitud coseno
    similitudes = cosine_similarity(modelo_render.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_render[modelo_render['id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_render.iloc[indices_juegos_similares]['app_name']
    #Obtengo un diccionario para la salida en la api
    #juegos_recomendados=juegos_recomendados.to_dict()
    return {
        "Juegos_recomendados" :juegos_recomendados}

In [65]:
recomendacion_juego(3310)

{'Juegos_recomendados': 105         Dynomite Deluxe
 103        Bejeweled Deluxe
 102         AstroPop Deluxe
 101             Zuma Deluxe
 100    Insaniquarium Deluxe
 Name: app_name, dtype: object}